In [ ]:
import pandas as pd
import plotly.express as px
import os
import numpy as np

# Directory containing the trading history CSV files
trading_dir = "trading/full_analysis"

# Get a list of all CSV files in the directory
## filter out random
csv_files = [f for f in os.listdir(trading_dir) if f.endswith(".csv") and not "random" in f]

# Create an empty list to store the dataframes
dfs = []

# Read data from each CSV file
for file in csv_files:
    file_path = os.path.join(trading_dir, file)
    df = pd.read_csv(file_path)
    df['Trading History'] = file  # Add a column to identify the trading history
    dfs.append(df)

# Combine all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

# Convert 'Total Value' to numeric, coercing errors
combined_df['Total Value'] = pd.to_numeric(combined_df['Total Value'], errors='coerce')

# Normalize 'Total Value' by dividing by the first value for each trading history
combined_df['Normalized Total Value'] = combined_df.groupby('Trading History')['Total Value'].transform(lambda x: x / x.mean())

# Plot the time series data with Plotly Express
fig = px.line(combined_df, x='Date', y='Total Value', color='Trading History',

              title='Trading History Comparison with Variance Bands')

# Update layout for better readability
fig.update_layout(xaxis_title='Date', yaxis_title='Total Value',

                  xaxis_tickangle=-45, title_x=0.5)

fig.show()
days = 60
# Calculate 14-day moving average standard deviation on normalized values, starting with 1 for the first days -1days
combined_df['Moving Avg Standard Deviation'] = combined_df.groupby('Trading History')['Normalized Total Value'].rolling(window=days).std().reset_index(level=0, drop=True)

# Fill the first days-1 NaN values (due to rolling window) with 1
combined_df['Moving Avg Standard Deviation'] = combined_df.groupby('Trading History')['Moving Avg Standard Deviation'].apply(lambda x: x.fillna(0, limit=days-1)).reset_index(level=0, drop=True)

fig_variance = px.line(combined_df.dropna(subset=['Moving Avg Standard Deviation']), x='Date', y='Moving Avg Standard Deviation', color='Trading History',
                       title=str(days)+'-Day Moving Average Standard Deviation of Normalized Total Value (Initial Standard Deviation = 1)')

fig_variance.update_layout(xaxis_title='Date', yaxis_title='Moving Average Standard Deviation (Normalized)',
                            xaxis_tickangle=-45, title_x=0.5)

fig_variance.show()


In [23]:
combined_df['Return'] = combined_df.groupby('Trading History')['Normalized Total Value'].pct_change()
combined_df['Z-Score Return'] = combined_df.groupby('Trading History')['Return'].transform(lambda x: (x - x.mean()) / x.std())
fig_zscore = px.line(combined_df.dropna(subset=['Z-Score Return']), 
                     x='Date', y='Z-Score Return', 
                     color='Trading History',
                     title='Z-Scored Returns (Mean=0, Std=1): Highlighting Gains & Losses')

fig_zscore.update_layout(xaxis_title='Date', yaxis_title='Z-Score Return',
                         xaxis_tickangle=-45, title_x=0.5)

fig_zscore.show()


In [19]:
extreme_events = combined_df[(combined_df['Z-Score Return'] > 2) | (combined_df['Z-Score Return'] < -2)]


In [24]:
import plotly.graph_objects as go

# Add markers for extreme losses (z-score < -2)
fig_zscore.add_trace(go.Scatter(
    x=extreme_events[extreme_events['Z-Score Return'] < -2]['Date'],
    y=extreme_events[extreme_events['Z-Score Return'] < -2]['Z-Score Return'],
    mode='markers',
    marker=dict(color='red', size=8, symbol='triangle-down'),
    name='Extreme Losses',
    text=extreme_events[extreme_events['Z-Score Return'] < -2]['Trading History'],
    hoverinfo='text+y'
))

# Add markers for extreme gains (z-score > 2)
fig_zscore.add_trace(go.Scatter(
    x=extreme_events[extreme_events['Z-Score Return'] > 2]['Date'],
    y=extreme_events[extreme_events['Z-Score Return'] > 2]['Z-Score Return'],
    mode='markers',
    marker=dict(color='green', size=8, symbol='triangle-up'),
    name='Extreme Gains',
    text=extreme_events[extreme_events['Z-Score Return'] > 2]['Trading History'],
    hoverinfo='text+y'
))
